In [1]:
from moviepy.editor import VideoFileClip

# Load the video file
video = VideoFileClip("/home/kush/CODE/Video/video.mp4")

# Extract the audio
audio = video.audio

# Save the audio as an mp3 file
audio.write_audiofile("output_audio.wav")

print("Audio extraction completed successfully!")


MoviePy - Writing audio in output_audio.wav


MoviePy - Done.
Audio extraction completed successfully!


In [2]:
!ffmpeg -y -i output_audio.wav -acodec pcm_s16le -ar 16000 outputAudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

NameError: name 'torch' is not defined

In [4]:
from huggingsound import SpeechRecognitionModel
import torch

# Initialize the device (CPU or GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the fine-tuned speech recognition model
model = SpeechRecognitionModel('facebook/wav2vec2-large-960h-lv60-self',device=device)
print(f"Loaded model on {device} successfully")


2024-10-22 21:55:06.755300: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 21:55:06.875590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 21:55:06.917271: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 21:55:06.928866: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 21:55:07.004959: I tensorflow/core/platform/cpu_feature_guar

10/22/2024 21:55:12 - INFO - huggingsound.speech_recognition.model - Loading model...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded model on cuda successfully


In [5]:
import librosa
input_file = 'outputAudio.wav'

In [6]:
print(librosa.get_samplerate(input_file))

stream = librosa.stream(
    input_file,
    block_length = 60,
    frame_length=16000,
    hop_length=16000
)

16000


In [7]:
import soundfile as sf
for i,speech in enumerate(stream):
    sf.write(f'{i}.wav',speech,16000)

In [8]:
audio_path = []
for a in range(i+1):
    audio_path.append(f'{a}.wav')

In [9]:
audio_path

['0.wav', '1.wav', '2.wav', '3.wav']

In [10]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 4/4 [00:10<00:00,  2.51s/it]


In [11]:
full_transcribe = ''

In [12]:

for item in transcriptions:
    full_transcribe += ''.join(item['transcription'])

In [13]:
full_transcribe

"FOR EVEN AS WE CELEBRATE TO NIGHT WE KNOW THE CHALLENGES THAT TO MORROW WILL BRING ARE THE GREATEST OF OUR LIFETIME I STAND HERE KNOWING THAT MY STORY IS PART OF THE LARGER AMERICAN STORY THAT I OWE A DEBT TO ALL THOSE WHO CAME BEFORE ME AND THAT IN NO OTHER COUNTRY ON EARTH IS MY STORY EVEN POSSIBLE I AM THE SON OF A BLACK MAN FROM KENYA AND A WHITE WOMAN FROM KANSAS I AM MARRIED TO A BLACK AMERICAN WHO CARRIES WITHIN HER THE BLOOD OF SLAVES AND SLAVE OWNERS I HAVE BROTHERS SISTERS NIECES NEPHEWS UNCLES AND COUSINS OF EVERY RACE AND EVERY HUE SCATTERED ACROSS THREE CONTINENTS THESE PEOPLE ARE PART OF ME AND THEY ARE PART OF AMERICA THIS COUNTRY THAT I LOVE I AVE COME HERE TO CAIROSEEK A NEW BEGINNING BETWEEN THE UNITED STATES AND MOSLEMS AROUND THE WORLD ONE BASED ON MUTUAL INTEREST AND MUTUAL RESPECT JUST AS MOSLEMS DO NOT FIT A CRUDE STEREOTYPE AMERICA IS NOT THE CRUDE STEREOTYPE OF A SELF INTERESTED EMPIRE TO SAY THAT FORCE MAY SOMETIMES BE NECESSARY IS NOT A CALL TO CYNICISM IT I

In [14]:
from transformers import pipeline

In [15]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [16]:
summarization(full_transcribe)

[{'summary_text': ' I stand here knowing that my story is part of the larger American American Story that I OWE A DEBT TO ALL THOSE WHO CAME BEFORE ME AND THAT IN NO OTHER COUNTRY ON EARTH IS MY STORY EVEN POSSIBLE I AM THE SON OF A BLACK MAN FROM KENYA AND A WHITE WOMAN FROM KANSAS I AM MARRIED TO A BLACK AMERICAN WHO CARRIES WITHIN HER THE BLOOD OF SLAVES AND SLAVE OWNERS I have BROTHERS SISTERS Sisters NIECES NEPHEWS UNCLES AND COUSINS of every RACE AND EVERY HUE SCATTERED AC'}]

In [18]:
from textblob import TextBlob
for item in transcriptions:
    text = summarization(item['transcription'])
    sentences = [item['summary_text'] for item in text]
    joined_text = ' '.join(sentences)    
    b = TextBlob(joined_text)
    print(b.correct())

 I stand here knowing that my story is part of the larger American Story that I OWE A DEBT of ALL THOSE WHO CAME BEFORE of AND THAT of of OTHER COUNTRY of EARTH of of STORY EVEN POSSIBLE I of THE SON of A BLACK MAN FROM KENYA AND A WHITE WOMAN FROM KANSAS I of MARRIED of A BLACK AMERICAN WHO CARRIES WITHIN HER the BLOOD of SLAVES AND SLAVE OWNERS .
 America is not the CRUDE STEREOTYPE of a SELF INTERESTED EMPIRE to say that FORCE MAY SOMETIMES be NECESSARY is not a call to CYNICISM . SEEK a new BEGINNING BETWELLING BETWEINS AND MOSLEMS AROUND THE world one based on MUTUAL INTEREST AND MUTAL RESPECT .


Your max_length is set to 142, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


 President Drama has been President of the U.S. for nearly two terms . He has said he is more optimistic about the future of America than ever before . The White House has been the subject of numerous scandals, including 9/11 attacks and 9/12 attacks .
 TENEVEPA AND B C NEWS FANS THANKs FOR CHECKING OUT OUR U TUBE CHANNEL SUBSCRIBE of CLICKING of THAT BUTTON DOWN HERE AND CLICK of ANY of THE VIDIOS OVER HERE of WATCH THE LATEST INTERVIEWS SHOW HIGH LIGHTS AND DIGITAL EXCLUSIVE .


In [19]:
from textblob import TextBlob

chunk_size = 6

for i in range(0, len(transcriptions), chunk_size):

    chunk = transcriptions[i:i + chunk_size]

    full_transcribe = ''.join([item['transcription'] for item in chunk])

    text = summarization(full_transcribe,max_length=100,min_length = 2,do_sample=False)
    sentences = [item['summary_text'] for item in text]

    joined_text = ' '.join(sentences)    
    b = TextBlob(joined_text)
    
    print(b.correct(),"/n/n")


 ForOKAA WON'T SANI AND YOUR DAD of WORRIED THAT YOU're GONE THIS LATE of NIGHT of YOU DON'T NEED of WORRY OKAZAKI I Old my PARENTS that I WAS GOING of of WITH of CHAN of NIGHT REALLY HOW DID your DAD REACT of THAT? I JUST WORRY about that when it HAPPENS HEY YOU'of BEING A LITTLE HASTY AREN'T YOUI CANT WAIT ANY MORE I SWEAR  SUCH A LOT of the TIMES of SEEMS TOO D
 The day after the WEDDING is the day after to MORROW. The BRIDE and GROOM AREN'T HERE of THEY HAVE of of HERE? I WONDER CAN of HAVE an ADVANCE CELEBRATION? I BET YOU'D JUST of HAPPY of SIT HERE ALL NIGHT WITH THAT HAT of YOUR HEAD YOU KNOW that WOL THAT'S NOT the ONLY THING that MAKES of HAPPy allTOGETHER. of CAN'T BELIEVE THAT YOU're in SUCH HIGH SPIRITS ALREADY THE WEDdING
 I REMEMBER MISSY BOKEY . It was BECAUSE of YOUR REQUEST THAT THIS WEDDING WILL of HELD of THE SCHOOL THAT'S RIGHT That'S WHAT of ISNO I FORGOT of TIN TIS of PORTANT TURCA I CAN'T REMEMBER WHY DID YOU AND I WANT of CELEBRATE CUCKO'S MARRIAGE? I CAN't rem

Your max_length is set to 130, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


 Miss CUCKO and her sister were always together and loved each other . The was the sister of Tissue BOKEY who was married to MISSIE BOKEY . Of we continue to be HAPPY she can be happy for her sister, she can also be happy .
 A LITTLE BIT but I really don't want to CRY even so if you have to by YOU'll CRY YOU'of CRY of TELL of ARE YOU A BOY TOO YES . A LDRIGHT: "I really want to cry a little bit. A little bit but I REALLY DON'T WANT of CRY EVEN of of YOU HAVE of CRRY you'll cry"
